In [2]:
import json
import pennylane as qml
import pennylane.numpy as np

In [135]:
# pennylane challenge the super parameter
# https://pennylane.ai/challenges/superparameter/

NUM_WIRES = 3

dev = qml.device("default.qubit", wires=NUM_WIRES)

@qml.qnode(dev)
def model(alpha):
    """In this qnode you will define your model in such a way that there is a single 
    parameter alpha which returns each of the basic states.

    Args:
        alpha (float): The only parameter of the model.

    Returns:
        (numpy.tensor): The probability vector of the resulting quantum state.
    """
    # uniform superposition of computational basis states
    for i in range(NUM_WIRES):
        qml.Hadamard(wires=i)

    # phase gates
    for i in range(NUM_WIRES):
        qml.RZ(alpha*np.pi/2**(i-1),wires=i)

    # inverse QFT convert phase shifts to addition in the computational basis
    qml.adjoint(qml.QFT)(wires=range(NUM_WIRES))
    
    
    return qml.probs()

In [136]:
def generate_coefficients():
    """This function must return a list of 8 different values of the parameter that
    generate the states 000, 001, 010, ..., 111, respectively, with your ansatz.

    Returns:
        (list(int)): A list of eight real numbers.
    """
    
    coef = np.zeros(2**NUM_WIRES)
    alpha = np.linspace(start=0,stop=10, num=5000)
    epsilon = 0.00001

    for a in alpha:
        for i in range(2**NUM_WIRES):
            probs = model(a)

            if abs(1-probs[i]) < epsilon:
                coef[i] = a


    return coef

In [137]:
coef = generate_coefficients()
output = [model(c) for c in coef]

for i in range(len(coef)):
    print(output[i][i],np.isclose(output[i][i],1))

0.9999979265554858 True
0.9999988336870405 True
0.9999994816385511 True
0.9999998704096176 True
0.9999999999999996 True
0.9999936500869518 True
0.9999953347546412 True
0.999996760244446 True


In [142]:
coef

tensor([ 7.99959992,  8.49969994,  8.99979996,  9.49989998, 10.        ,
         6.49929986,  6.99939988,  7.4994999 ], requires_grad=True)

In [139]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    return None

def check(solution_output, expected_output: str) -> None:
    coefs = generate_coefficients()
    output = np.array([model(c) for c in coefs])
    epsilon = 0.001

    for i in range(len(coefs)):
        assert np.isclose(output[i][i], 1)

    def is_continuous(function, point):
        limit = calculate_limit(function, point)

        if limit is not None and sum(abs(limit - function(point))) < epsilon:
            return True
        else:
            return False

    def is_continuous_in_interval(function, interval):
        for point in interval:
            if not is_continuous(function, point):
                return False
        return True

    def calculate_limit(function, point):
        x_values = [point - epsilon, point, point + epsilon]
        y_values = [function(x) for x in x_values]
        average = sum(y_values) / len(y_values)

        return average

    assert is_continuous_in_interval(model, np.arange(0,10,0.001))

    for coef in coefs:
        assert coef >= 0 and coef <= 10

In [140]:
# These are the public test cases
test_cases = [
    ('No input', 'No output')
]

In [141]:
# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input 'No input'...
Correct!
